# Scrape Documents

This Scraper ask politely for all protocols (pdfs) and stores them.  

Ressources:
* https://www.zh.ch/de/politik-staat/opendata.html?keyword=ogd#/details/709@fachstelle-ogd-kanton-zuerich
* https://github.com/openZH/KRZH_gever
* https://parlzhcdws.cmicloud.ch/parlzh5/index/geschaeft/index.html

In [1]:
import urllib
import calendar
import requests
from pathlib import Path
import glob, os
import xml.etree.ElementTree as ET
import copy
import pandas as pd

## 1. Load all Meetings (Sitzungen)

In [2]:
url = 'https://parlzhcdws.cmicloud.ch/parlzh3/cdws/Index/SITZUNGENDETAIL/searchdetails?q=%s&l=de-CH'

year_from = 2010
year_to = 2022
print("Years: %s - %s" % (year_from, year_to))
for year in range(year_from, year_to):
    for i in range(0, 3):
        #print("%s %s" % (year, i))

        # First day
        d_start = '%s-%02d-01 00:00:00' % (year, i * 4 + 1)

        # Last day
        l = calendar.monthrange(year, (i + 1) * 4)[-1]
        d_end = '%s-%02d-%s 23:59:00' % (year, (i + 1) * 4, l)

        q = 'datum_start >= "%s" and datum_start <= "%s" sortBy datum_end/sort.ascending uhrzeit/sort.ascending' % (d_start, d_end)

        r = requests.get(url % urllib.parse.quote(q))
        with open(Path('../export/SITZUNGENDETAIL/%s_%s.xml' % (year, i)), 'wb') as f:
            f.write(r.content)

Years: 2010 - 2022


## 2. Create DataFrame from XML

In [3]:
# Parse all XML and create Dataframe
ns = {
    's': 'http://www.cmiag.ch/cdws/searchDetailResponse',
    'd': 'http://www.cmiag.ch/cdws/SitzungenDetail'
    }

records = []

# Open all xml-files
for f in glob.glob(str(Path('../export/SITZUNGENDETAIL/*.xml'))):
    tree = ET.parse(f)
    #tree = ET.parse('../export/SITZUNGENDETAIL/2021_1.xml')
    root = tree.getroot()

    for sitzung in root.findall('.//s:Hit/d:Sitzung', ns):
        sitzungsdokumente = sitzung.findall('d:Sitzungsdokumente/d:Dokument', ns)
        
        # Skip if no docs
        if len(sitzungsdokumente) == 0:
            continue
        
        #print(sitzungsdokumente)

        record = {
            'sitzung_id': sitzung.attrib['OBJ_GUID'],
            'sitzung_name': sitzung.find('d:Titel', ns).text,
            'sitzung_date': sitzung.find('d:Datum/d:Start', ns).text,
            'sitzung_gremium': sitzung.find('d:Gremium/d:Gremium/d:Kurzname', ns).text
        }

        #if record['sitzung_id'] != '5ffba37035f1487fbf889a3f26c7ae9c': continue

        # Loop each Doc
        for doc in sitzungsdokumente:

            r = copy.copy(record)
            r['dokument_id'] = doc.attrib['OBJ_GUID']
            r['dokument_titel'] = doc.find('d:Titel', ns).text
            r['dokument_kategorie'] = doc.find('d:Kategorie', ns).text

            # Get eDoc. Assume, there is only one "eDokument" as child of Dokument
            edoc = doc.find('d:eDokument', ns)

            # Check
            if len(doc.findall('d:eDokument', ns)) > 1:
                raise ValueError('More than one eDoc')

            r['eDokument_id'] = edoc.attrib['ID']
            r['eDokument_filename'] = edoc.attrib['FileName']
            r['version_nr'] = edoc.findall('d:Version', ns)[-1].attrib['Nr']

            records.append(r)

# Create DataFrame
df = pd.DataFrame(records)

df['sitzung_date'] = pd.to_datetime(df['sitzung_date'])

# Create Filename
def create_filename(row):
    return "%s-%s.pdf" % (
        row['sitzung_date'].strftime('%Y-%m-%d'),
        row['eDokument_id']
    )
df['_filename'] = df.apply(create_filename, axis=1)

# Sort and Store
df = df.sort_values('sitzung_date')
df.to_csv(Path('../export/dokumente.csv'), index=False)

## 3. Download Documents

In [4]:
df = pd.read_csv(Path('../export/dokumente.csv'))

url = 'https://parlzhcdws.cmicloud.ch/parlzh3/cdws/Files/%s/%s/pdf'

# Select only "Protokoll"
for i, row in df[df.dokument_kategorie == 'Protokoll'].iterrows():
    
    fname = Path('../export/Files/%s' % row['_filename'])

    # File exists?
    if os.path.isfile(fname):
        continue

    r = requests.get(url % (row['eDokument_id'], row['version_nr']))
    if r.status_code != 200:
        raise ValueError('File not found', row['eDokument_id'], row['version_nr'])

    with open(fname, 'wb') as f:
        f.write(r.content)

print("finito")

finito
